In [4]:
from my_trader import *
import pandas as pd
import numpy as np
from selenium import webdriver
import urllib2
import time
import re
import getpass

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


def create_driver_session(session_id, executor_url):
    from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver

    # Save the original function, so we can revert our patch
    org_command_execute = RemoteWebDriver.execute

    def new_command_execute(self, command, params=None):
        if command == "newSession":
            # Mock the response
            return {'success': 0, 'value': None, 'sessionId': session_id}
        else:
            return org_command_execute(self, command, params)

    # Patch the function before creating the driver object
    RemoteWebDriver.execute = new_command_execute

    new_driver = webdriver.Remote(command_executor=executor_url, desired_capabilities={})
    new_driver.session_id = session_id

    # Replace the patched function with original function
    RemoteWebDriver.execute = org_command_execute

    return new_driver
#---------------------------
def render_text_contact(get_temp):
    flag = False
    trial =0
    result = pd.DataFrame()
    while not flag:
        try:
            get_temp = get_temp.text.split("\n")
            flag = True
        except Exception as e:
            print "Get text problem"
            if trial > 3:
                result = [np.NaN,np.NaN,np.NaN,np.NaN]
                return result
            driver.refresh()
            time.sleep(10+2*trial)
            trial +=1
            #get_temp = get_temp.get_attribute('innerHTML')
            #print ("!!!! verify: " + get_temp)
            #time.sleep(step_wait)
            #flag= True
            
    header=["name","position","location","industry"]
    count_header = 0
    #get_temp = get_temp.text.split(":")
    for i in range(len(get_temp)):
        get_temp[i] = get_temp[i].encode('ascii','ignore')
        if ":" in get_temp[i]:
            #flag = True
            get = get_temp[i].split(":")
            result = result.append([get])

        else:
            result = result.append([[header[count_header],get_temp[i]]])
            count_header +=1

    result = result.reindex()    
    return result

def get_progress(name):
    mongodb = mongo("hoover")
    progress = pd.DataFrame(list(mongodb.db[name].find())).company_name
    progress = list(progress)
    return progress
#---------------------------
mongodb = mongo("hoover")



In [11]:
username = raw_input()
password = getpass.getpass()

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [3]:

#------PARAMETERS---------------

step_wait = 2
page_wait = 5
collection_name = "Finance and Technology"
list_row_number = 3  #input the list position of the list you want to crawl, starting 0

progress = get_progress(collection_name)


#flag initialize
from_beginning =  True

rebot_flag = True #default to be True to start with the program 

just_open_window = False  #default to be False to start with the program 
#-------------------------------

In [37]:

if rebot_flag:  
    #reset flag
    rebot_flag = False
#===============OPEN BROWER=============================================
    driver=webdriver.Firefox()
    session_id =driver.session_id
    executor_url =driver.command_executor._url
    driver.implicitly_wait(5)

    driver.get('https://signon.onesource.com/Web/Logon/Signin.aspx')
    time.sleep(step_wait)
    driver.implicitly_wait(5)
    driver.find_element_by_id("username").clear()
    driver.find_element_by_id("username").send_keys(username)
    driver.find_element_by_id("password").clear()
    driver.find_element_by_id("password").send_keys(password)
    driver.find_element_by_id("btnSignIn").click()
    time.sleep(step_wait)
    driver.implicitly_wait(10)
    driver.find_element_by_link_text("Lists").click()
    time.sleep(step_wait)
    driver.implicitly_wait(10)

    # list selector

    list_ele = driver.find_elements_by_class_name("name-row")
    get = list_ele[list_row_number]
    get.find_element_by_tag_name("a").click()



    #page info

    current_page = driver.find_element_by_class_name("current-page")
    current_page=current_page.get_attribute('innerHTML')
    current_page = int(current_page)

    total_page = driver.find_element_by_class_name("total-page")
    total_page=total_page.get_attribute('innerHTML')
    total_page = int(total_page)
#================================================================================
if just_open_window:
    raise SystemExit(0)
while True:
    try:
        driver.implicitly_wait(100)
        current_page = driver.find_element_by_class_name("current-page")
        current_page=current_page.get_attribute('innerHTML')
        current_page = int(current_page)

        # company selector
        company_list = driver.find_elements_by_class_name("clickable")
        name_list=[]
        for i in range(len(company_list)):
            name_list.append(company_list[i].text)    

        for i in name_list:
            #clear_output()

            #company_list = driver.find_elements_by_class_name("clickable")        
            #get_company = company_list[i]
            if i in progress:
                print (i+ " already in database, skipping")
                continue
            page_trial = 0
            while True:
                try: 
                    if page_trial <3:
                        time.sleep(page_wait)
                        driver.implicitly_wait(10)   
                        get_company = driver.find_element_by_link_text(i)
                        driver.execute_script("return arguments[0].scrollIntoView();", get_company)
                        driver.execute_script("arguments[0].click();", get_company)
                        #company_list[i].click()
                        WebDriverWait(driver, 80).until(EC.presence_of_element_located((By.ID, "totalContacts")))
                        time.sleep(step_wait)
                        #get basic information
                        info = {}
                        address = driver.find_element_by_class_name("company-address").text
                        company_name = driver.find_element_by_class_name("name").text
                        try:
                            website = driver.find_element_by_class_name("url").text
                            info["website"] = website
                        except:
                            print ("no website found")
                            info["website"]=np.NaN
                        data = driver.find_elements_by_class_name("data-container")
                        for i in range(len(data)):

                            try:
                                label=data[i].find_element_by_class_name("data-label")
                            except:
                                continue 
                                #label=data[i].find_element_by_class_name("data-label wide")
                            value=data[i].find_elements_by_class_name("data-value")
                            value = value[0].text
                            label = label.text
                            label = label.encode('ascii','ignore')
                            info[label] = value

                        info["address"] = address
                        info["company_name"] = company_name

                        mongodb.db[collection_name].insert_one(info)
                        print (company_name + ": basic information got")
                        #get description about the company

                        data_title = ["description","industry","highlights"]
                        description=pd.DataFrame()
                        for i in data_title:
                            get_temp = driver.find_element_by_class_name("highlights")
                            get_temp = get_temp.text.split("\n")
                            #get_temp = get_temp.text.split(":")
                        for i in range(len(get_temp)):
                            get_temp[i] = get_temp[i].encode('ascii','ignore')
                            get = get_temp[i].split(":")
                            description = description.append([get])
                        description = description.set_index(0)
                        description=json.loads(description.to_json()).values()[0]
                        mongodb.db[collection_name].update_one({"company_name":company_name},{"$set":description},upsert=True)
                        print (company_name + ": description information got")
                        break
                    else:
                        break
                except Exception as e:
                    print ("----page_info----")
                    print e
                    print ("---------------")
                    driver.back()
                    driver.forward()
                    page_trial +=1            

            #Get contacts
            contacts_flag = False
            contact_trial = 0 
            while not contacts_flag:
                try:
                    contacts = driver.find_element_by_id("totalContacts").text
                    contacts = int(contacts)
                    contacts_flag = True
                except:
                    #print ("contact number didn't come out yet")
                    continue
            while True:
                try:
                    if contacts != 0 and contact_trial<3:
                        try:
                            driver.find_element_by_id("company_contacts").click()
                        except:
                            driver.execute_script("arguments[0].click();", driver.find_element_by_id("company_contacts"))
                        time.sleep(step_wait)
                        WebDriverWait(driver, step_wait).until(EC.presence_of_element_located((By.CLASS_NAME, "detail-container")))                                   
                        contacts = driver.find_elements_by_class_name("detail-container")

                        contacts = driver.find_elements_by_class_name("search-result-container")
                        result = []
                        for i in range(len(contacts)):

                            temp = render_text_contact(contacts[i])
                            temp = temp.set_index(temp[0])
                            temp = temp.drop(0,axis=1)
                            result.append(json.loads(temp.to_json()).values())
                        mongodb.db[collection_name].update_one({"company_name":company_name},{"$set":{"contact":result}},upsert=True)
                        print (company_name + ": contacts information got")
                        driver.back()
                        break
                    else:
                        print ("no contact available")
                        break

                # page wait time   

                except Exception as e:
                    print ("--contact------")
                    print e
                    print ("---------------")
                    driver.back()
                    driver.forward()
                    contact_trial +=1

            #finished one company
            progress.append(company_name)
            driver.back()
    except Exception as e:
        print ("--back to list------")
        print e
        print ("---------------")
        driver.refresh()
        driver.back()
            
        # next page
    if from_beginning:
        if total_page > current_page:
            next_page = driver.find_element_by_class_name("next-page")
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(page_wait)
            driver.implicitly_wait(10) 
        else:
            break
    else:
        if current_page>80:
            #print "previous page"
            last_page = driver.find_element_by_class_name("previous-page")
            driver.execute_script("arguments[0].click();", last_page)
            time.sleep(page_wait)
            driver.implicitly_wait(10) 
        else:
            break

#================================================    
#Getting hoover data


hoover_data = pd.DataFrame(list(mongodb.db[collection_name].find())).drop("",axis = 1)
hoover_data=hoover_data.set_index("company_name")
hoover_contact_temp=hoover_data.loc[:,"contact"].dropna()
#Getting hoover contact data
hoover_contacts = pd.DataFrame()
for i in range(len(hoover_contact_temp)):
    for j in hoover_contact_temp.iloc[i]:
        temp = pd.DataFrame(j,index=[hoover_contact_temp.index[i]])
        hoover_contacts = hoover_contacts.append(temp)

hoover_contacts.index.name = "company_name"
hoover_data.to_csv(collection_name + ".csv")
hoover_contacts.to_csv(collection_name+"_contacts.csv")

Grand Terrace Market already in database, skipping
Shop N Go Food Store already in database, skipping
USA Donuts already in database, skipping
Teresa's Market already in database, skipping
Texas Donuts already in database, skipping
Sugar Planet Inc already in database, skipping
Tony's Donuts already in database, skipping
Shinta Kawahara Company already in database, skipping
Playa Azul Tortilleria already in database, skipping
City Hall Market already in database, skipping
Simple Treats, LLC already in database, skipping
The Pit Stop already in database, skipping
Un Rancho Alegre Corporation already in database, skipping
Sun Harvest Salt LLC already in database, skipping
Boparai Farms already in database, skipping
La Penca Azul already in database, skipping
Mark Evans Farming Inc. already in database, skipping
Mavenwork Inc. already in database, skipping
Ojai East End Orchard already in database, skipping
Kyu's Market already in database, skipping
Benny Acosta already in database, skipp

Lakta Nutrition, Inc. already in database, skipping
Haiticoffee Co Inc already in database, skipping
Yutopian Enterprises Inc already in database, skipping
Imperial Spice Co already in database, skipping
Golden Cream Donuts No 2 already in database, skipping
Los Robles Market already in database, skipping
Asia Market already in database, skipping
Donut Star already in database, skipping
La Canada Market already in database, skipping
La Union Market already in database, skipping
Herbalife Armando & Claudia Distributors already in database, skipping
Patty Norton already in database, skipping
Imaging Royalty already in database, skipping
Singh Mandhir already in database, skipping
Won OH already in database, skipping
Rapha Beverage, Inc. already in database, skipping
Jsk Company already in database, skipping
Starlight Donuts already in database, skipping
Rossetti's Corner already in database, skipping
Stephs Donut Hole already in database, skipping
Jolly Donuts already in database, skippi

Lavictoria A Market already in database, skipping
Daily Donut already in database, skipping
Fosters Donuts already in database, skipping
Dolly's Donuts already in database, skipping
Daily Donuts already in database, skipping
Lucky Meat Co already in database, skipping
Pancho Meat Market already in database, skipping
Dn Top Donuts already in database, skipping
Master's Donuts already in database, skipping
Comus International already in database, skipping
Donuts Plus already in database, skipping
Donut Basket already in database, skipping
Bon Bon already in database, skipping
Frank's Market already in database, skipping
A-4 Market already in database, skipping
Universal Dairy already in database, skipping
Citrus Drops already in database, skipping
Dionne McBride already in database, skipping
Balbir Singh and Harminder Singh already in database, skipping
Fine Real Edibles Sow Happiness already in database, skipping
Gerardo S Gomez already in database, skipping
Javier Garcia already in dat

Beach Health Tree already in database, skipping
Fifty First Street Produce already in database, skipping
Teslo Industries Inc. already in database, skipping
Activalley LLC already in database, skipping
Ramiro A Ramirez already in database, skipping
HI Ho Market and Liquor already in database, skipping
Coffee Time already in database, skipping
La Feria Market already in database, skipping
Jax Donuts already in database, skipping
South Pacific Market already in database, skipping
Village Donuts already in database, skipping
Royal Oaks Farm already in database, skipping
Lee, Phong Market already in database, skipping
German Grupo already in database, skipping
Freshh' Donuts already in database, skipping
P & G Market already in database, skipping
David Cheng already in database, skipping
Hosanna Oriental Food already in database, skipping
Centro De Nutricion already in database, skipping
Lucy Bakery already in database, skipping
Sk's Donut & Croissant already in database, skipping
Robert's

Hom's Grocery already in database, skipping
Evergreen Donut already in database, skipping
Donut Inn already in database, skipping
The Beanery already in database, skipping
El Tronquito Market already in database, skipping
Mina's Meat Market already in database, skipping
Rosa Lavella Bakery already in database, skipping
Carniceria 3 Hermanos already in database, skipping
Jubilee Donuts already in database, skipping
L&H Cake's & Party Supplies already in database, skipping
Krispy's Donuts already in database, skipping
Donut House already in database, skipping
Lyon's International already in database, skipping
Scatenas Family Trust already in database, skipping
Oliveira Bros already in database, skipping
Sugar Whitlock Studio already in database, skipping
Mann, Awtar already in database, skipping
Samara Ranches, LLC already in database, skipping
Nikan Orchards, LLC already in database, skipping
Joe R Garcia already in database, skipping
Kanase, Kyoko already in database, skipping
Jaspal O

no website found
Bob's Tiny Market: basic information got
Bob's Tiny Market: description information got
no contact available
no website found
Gardena Fish Market Corp.: basic information got
Gardena Fish Market Corp.: description information got
Gardena Fish Market Corp.: contacts information got
no website found
Donut World: basic information got
Donut World: description information got
Donut World: contacts information got
no website found
Chinese Bakery: basic information got
Chinese Bakery: description information got
Chinese Bakery: contacts information got
no website found
Vista Doughnuts: basic information got
Vista Doughnuts: description information got
Vista Doughnuts: contacts information got
Sogo Bakery, Inc. already in database, skipping
Jelly Donut already in database, skipping
no website found
Dhaliwal Brothers Farms: basic information got
Dhaliwal Brothers Farms: description information got
Dhaliwal Brothers Farms: contacts information got
Frank's Distributing of Produc

Roosevelt Bakery: contacts information got
no website found
Betsy's Cake Center: basic information got
Betsy's Cake Center: description information got
Betsy's Cake Center: contacts information got
no website found
Angel's Health Foods Center: basic information got
Angel's Health Foods Center: description information got
no contact available
no website found
Santa Rosa Market: basic information got
Santa Rosa Market: description information got
no contact available
no website found
Parkway Mini Mart: basic information got
Parkway Mini Mart: description information got
no contact available
no website found
Tita's Bakery: basic information got
Tita's Bakery: description information got
Tita's Bakery: contacts information got
no website found
Freshstax: basic information got
Freshstax: description information got
no contact available
no website found
Mercado Mexico: basic information got
Mercado Mexico: description information got
Mercado Mexico: contacts information got
no website found


Yu-Lee, Inc.: description information got
Yu-Lee, Inc.: contacts information got
no website found
Laestrella Market: basic information got
Laestrella Market: description information got
Laestrella Market: contacts information got
Asian Market already in database, skipping
no website found
Longchang Market: basic information got
Longchang Market: description information got
no contact available
no website found
Bread Deluxe Bakery: basic information got
Bread Deluxe Bakery: description information got
Bread Deluxe Bakery: contacts information got
no website found
G & J Market: basic information got
G & J Market: description information got
G & J Market: contacts information got
Har Bin Deer Antlers Trading Co: basic information got
Har Bin Deer Antlers Trading Co: description information got
no contact available
Carnita's Uruapan already in database, skipping
Pam's Donuts already in database, skipping
Cub Systems already in database, skipping
Market Insights Consulting already in databa

Gomez Produce Corporation already in database, skipping
La Unica Bakery & Tortillia already in database, skipping
Cypress Drive-In Dairy already in database, skipping
Market 1 already in database, skipping
Mike Hikmat already in database, skipping
Jmac Trading, Inc. already in database, skipping
Muranaka Farm already in database, skipping
Family Nutrition, Inc. already in database, skipping
Wang's Store already in database, skipping
Tropicana Delite already in database, skipping
Sonias Bakery already in database, skipping
Sun Valley Wholesale Ice Cream & Candy already in database, skipping
Classic Doughnuts & Croissants already in database, skipping
Asian City Development Inc already in database, skipping
Quik Stop already in database, skipping
Rai, Ravinderpal already in database, skipping
Warrior Nutrition, LLC already in database, skipping
Joe Maria Company Inc already in database, skipping
California Dry Fruits already in database, skipping
Aguilar Family Trust already in database,

In [4]:
hoover_data = pd.DataFrame(list(mongodb.db[collection_name].find())).drop("",axis = 1)
hoover_data=hoover_data.set_index("company_name")
hoover_contact_temp=hoover_data.loc[:,"contact"].dropna()
#Getting hoover contact data
hoover_contacts = pd.DataFrame()
for i in range(len(hoover_contact_temp)):
    for j in hoover_contact_temp.iloc[i]:
        temp = pd.DataFrame(j,index=[hoover_contact_temp.index[i]])
        hoover_contacts = hoover_contacts.append(temp)

hoover_contacts.index.name = "company_name"
hoover_data.to_csv(collection_name + ".csv")
hoover_contacts.to_csv(collection_name+"_contacts.csv")